In [1]:
import scipy.io as spio
import numpy as np
import os

In [ ]:
path = '../data/Processed/2021/01/01'

files = [folder for folder in os.listdir(path)]

In [2]:
lib = spio.loadmat('master_program\Inputs\covis_bathy_2019b.mat')
covis = spio.loadmat("../data/Processed/2021/01/03/COVIS-20210103T000002-fullimaging1/COVIS-20210103T000002-imaging1.mat")

xg = covis["imaging"]["grid"][0][0][0]["x"][0]
yg = covis["imaging"]["grid"][0][0][0]["y"][0]
zg = covis["imaging"]["grid"][0][0][0]["z"][0]
vg = covis["imaging"]["grid"][0][0][0]["Id_filt"][0]

xb = lib["covis"]["grid"][0][0][0]["x"][0]
yb = lib["covis"]["grid"][0][0][0]["y"][0]
zb = lib["covis"]["grid"][0][0][0]["v"][0]
rb = np.sqrt(xb**2 + yb**2)
zb[(rb<4)] = np.NaN


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# %matplotlib ipympl
%matplotlib widget


# Plot 3D image
fig = plt.figure(1)
ax = fig.add_subplot(111, projection=Axes3D.name)
colour = "terrain_r"
color = "pink_r"
# Add bathy
pbathy = ax.plot_surface(xb, yb, zb, cmap=colour, vmin=-2, vmax=4, edgecolor='none', alpha=0.5)
# pbathy.set_cmap(colour)
pbathy.set_edgecolor('none')
pbathy.set_alpha(0.9)

ax.contour3D(xb, yb, zb, levels=np.arange(-2, 4, 0.2), colors="k", linestyles="solid", alpha=1)

# Set isosurface properties
isosurfs = [
    {'color': [77/255, 153/255, 204/255], 'value': -60, 'alpha': 0.1},
    {'color': [128/255, 77/255, 128/255], 'value': -50, 'alpha': 0.2},
    {'color': [153/255, 5/255, 13/255], 'value': -40, 'alpha': 0.3}
]

for isosurf in isosurfs:
    surf_value = isosurf['value']
    surf_color = isosurf['color']
    surf_alpha = isosurf['alpha']
    p = ax.plot_trisurf(xg.flatten(), yg.flatten(), zg.flatten(), cmap='viridis',
                        vmin=surf_value, vmax=np.max(zb), color=surf_color, alpha=surf_alpha, edgecolor='none')

# Set plot properties
# ax.set_xlim([-20, 1])
# ax.set_ylim([-8, 12])
# ax.set_zlim([-2, 15])

ax.set_xlim([-30, 1])
ax.set_ylim([-10, 30])
ax.set_zlim([-2, 15])

ax.set_xlabel('Easting of COVIS (m)')
ax.set_ylabel('Northing of COVIS (m)')
ax.set_zlabel('Height above COVIS base (m)')

ax.plot3D([0, 0], [0, 0], [0, 4.2], 'y', linewidth=4)
plt.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Set isosurface properties
isosurfs = [
    {'color': [77/255, 153/255, 204/255], 'value': -60, 'alpha': 0.1},
    {'color': [128/255, 77/255, 128/255], 'value': -50, 'alpha': 0.2},
    {'color': [153/255, 5/255, 13/255], 'value': -40, 'alpha': 0.3}
]

fig = go.Figure(data=go.Isosurface(
    x=xg.flatten(),
    y=yg.flatten(),
    z=zg.flatten(),
    value=vg.flatten()
    ))
fig.show()

In [4]:
import plotly.graph_objects as go
import numpy as np
from skimage import measure


isosurfs = [
    {"color": [77, 153, 204], "value": -60, "alpha": 0.1},
    {"color": [128, 77, 128], "value": -50, "alpha": 0.2},
    {"color": [153, 5, 13], "value": -40, "alpha": 0.3}
]

# Create the isosurface traces
isosurface_traces = []
for isosurf in isosurfs:
    v = vg.copy()
    eps = np.nan
    m = np.where(v == 0)
    v[m] = eps  # remove zeros
    v = 10 * np.log10(v)
    surf_value = isosurf["value"]
    surf_color = [c / 255 for c in isosurf["color"]]
    surf_alpha = isosurf["alpha"]

    verts, faces, _, _ =  measure.marching_cubes(v, level=surf_value)

    isosurface_traces.append(go.Mesh3d(
        x=verts[:, 0],
        y=verts[:, 1],
        z=verts[:, 2],
        opacity=surf_alpha,
        color=surf_color,
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        intensity=vg.flatten(),
        colorscale="Viridis",
        showscale=False
    ))

# Create the main isosurface plot
isosurface_plot = go.Figure(data=isosurface_traces)

# Update layout if needed
isosurface_plot.update_layout(scene=dict(
    xaxis_title='X-Axis Label',
    yaxis_title='Y-Axis Label',
    zaxis_title='Z-Axis Label'
))

# Show the plot
isosurface_plot.show()


ValueError: 
    Invalid value of type 'builtins.list' received for the 'color' property of mesh3d
        Received value: [0.30196078431372547, 0.6, 0.8]

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, darkslategrey,
            darkturquoise, darkviolet, d

In [3]:
import plotly.graph_objects as go

fig = go.Figure(go.Surface(
    contours = {
        "x": {"show": True, "start": -2, "end": 4, "size": 0.2, "color":"white"},
        "z": {"show": True, "start": -2, "end": 4, "size": 0.2}
    },
    x = xb[0,:],
    y = yb[:,0],
    z = zb
    ))

# Update the layout to set axis labels
fig.update_layout(scene=dict(
    xaxis_title='Easting of COVIS (m)',
    yaxis_title='Northing of COVIS (m)',
    zaxis_title='Height above COVIS base (m)'
))
fig.add_trace(go.Scatter3d(
    x=[0, 0],
    y=[0, 0],
    z=[0, 4.2],
    mode='lines',
    line=dict(color='yellow', width=4)
))

fig.show()